# Predicting Reservoir Storage

## Introduction
My project's goal is to model and predic reservoir water levels in California. Using a previous years temperature and precipitation data, I hoped to see how accurately I could predict reservoir capacity and provide a range of possible values, not given the current water year data.

This project is of utmost importance given California's intensifying climate crisis. For my process of data analysis and model training, I use Folsom Lake, the 10th largest dam in California and a key water supply for millions of its residents. My goal was to create a model that could use exogenous seasonal variables to predict seasonal water supply and its annual minimum.  

Folsom Lake is going increasingly deep in its reserves during the extending droughts, having got to as low as 14% of its capacity in 2014. My ultimate goal is to predict the conditions that would cause Folsom Lake to be unusable, ie. near empty and unavailable for use as a water supply for the millions of people who rely on it, the agricultural companies, and the water-species. 

In [1]:
#import libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA', FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA', FutureWarning)

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit




### The Data

To predict Folsom Lake's water storage, we can take daily data from the US Bureau of Reclamation. There is also an exogenous variable of interest which could help in predictions: the daily evaporation is available for each reservoir.

In [2]:
storage = pd.read_csv('data/folsomlake/folsom_storage.csv',header=7) #acre feet
evaporation = pd.read_csv('data/folsomlake/folsom_evaporation.csv',header=7) #sum cfs 24 hr

The 8-station Sierra Index tracks precipitation across the Northern Sierras, and is an accurate measure of the water coming into the Sacramento River watershed. The Sacramento River watershed is the largest watershed in California, and contains many of the key reservoirs we will be modelling.

Meanwhile, the 5-station Sierra Index tracks precipitation in the Southern Sierras, and is a measure of the water year of the San Joaquin River watershed, the second largest watershed in the state.

I downloaded these to utilize as an exogenous variable.

In [3]:
northsierra = pd.read_csv('data/8SI.csv')
southsierra = pd.read_excel('data/5SI.xlsx')

northsierra.rename({'Unnamed: 0':'date'},axis=1,inplace=True)

In [4]:
southsierra.tail()

,STATION_ID,DURATION,SENSOR_NUMBER,SENS_TYPE,DATE TIME,OBS DATE,VALUE,DATA_FLAG,UNITS
379,5SI,M,2,RAIN,NaN,20210801,0.04,NaN,INCHES
380,5SI,M,2,RAIN,NaN,20210901,0.10,NaN,INCHES
381,5SI,M,2,RAIN,NaN,20211001,6.61,NaN,INCHES
382,5SI,M,2,RAIN,NaN,20211101,NaN,NaN,INCHES
383,5SI,M,2,RAIN,NaN,20211201,NaN,NaN,INCHES


### Data Preprocessing

Editing the dataframes so they can be interpreted as time series and have matching start and end dates is crucial to our modelling. As water years begin October 1st each year, we will starting in October 1st 1990.

In [5]:
#drop unnecessary columns

drop_list = ['Location','Parameter','Timestep','Aggregation','Units']

storage.drop(drop_list,axis=1,inplace=True)
evaporation.drop(drop_list,axis=1,inplace=True)

In [6]:
#adjust dates
start_date = "1990-10-01"
end_date = "2021-10-01"

storage = storage[(storage['Datetime (UTC)'] >= start_date) & (storage['Datetime (UTC)'] <= end_date)]

evaporation = evaporation[(evaporation['Datetime (UTC)'] >= start_date) & (evaporation['Datetime (UTC)'] <= end_date)]

storage.tail()

,Result,Datetime (UTC)
11675,231948,2021-09-26 08:00:00
11676,231852,2021-09-27 08:00:00
11677,231138,2021-09-28 08:00:00
11678,230527,2021-09-29 08:00:00
11679,229541,2021-09-30 08:00:00


In [7]:
#convert north sierra and south sierra indices to time series data
northsierra['date'] = pd.to_datetime(northsierra['date'])
northsierra.set_index('date',inplace=True)

drop_list = ['STATION_ID','DURATION','SENSOR_NUMBER','SENS_TYPE','DATE TIME','DATA_FLAG','UNITS']
southsierra.drop(drop_list,axis=1,inplace=True)
southsierra.tail(15)

southsierra['OBS DATE'] = southsierra['OBS DATE'].apply(lambda x: pd.to_datetime(str(x),format='%Y%m%d'))

southsierra['OBS DATE'] 

0     1990-01-01
1     1990-02-01
2     1990-03-01
3     1990-04-01
4     1990-05-01
         ...    
379   2021-08-01
380   2021-09-01
381   2021-10-01
382   2021-11-01
383   2021-12-01
Name: OBS DATE, Length: 384, dtype: datetime64[ns]

In [11]:
list(range(1990,2021))

[1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020]